In [1]:
# %reload_ext autoreload
# %autoreload 2

# import os
# import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

# print(two_levels_up_directory)

# sys.path.append(two_levels_up_directory)

In [2]:
# !pip install pyngrok
# !pip install mlflow
# !pip install boto3

In [3]:
import pandas as pd

from pykoi.rlhf import RLHFConfig
from pykoi.rlhf import SupervisedFinetuning
import mlflow
import datetime
import torch
torch.cuda.empty_cache()

/home/ubuntu/miniconda3/envs/koi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [4]:
input_root = "input/"
input_file = "rlhf_qa_dataset.json"
stack_exchange_df = pd.read_json(input_root + input_file)
stack_exchange_df.head()

,Question,Answer
0,What is InstructGPT?,InstructGPT is a language model developed by O...
1,Why does InstructGPT work?,InstructGPT works due to a two-step training p...
2,What are some commonly used evaluation metrics...,One main evaluation metric for InstructGPT is ...
3,How is InstructGPT used?,InstructGPT can be used in any application tha...
4,What are some common applications of InstructGPT?,Common applications of InstructGPT can be in e...


In [5]:
from datasets import Dataset

print("My ranking database has {} samples".format(stack_exchange_df.shape[0]))
dataset = Dataset.from_dict(stack_exchange_df)
dataset

My ranking database has 45 samples


Dataset({
    features: ['Question', 'Answer'],
    num_rows: 45
})

### Train with RLHF

In [6]:
# Set up mlflow experiment name.

# mlflow.set_tracking_uri("http://x.x.x.x:5000")
experiment = "rlhf_step2_reward"
current_time = str(datetime.datetime.now())
mlflow_experiment_name = '/'.join([experiment, current_time])

try:
    mlflow.end_run()
except:
    print("No mlflow run in progress")

mlflow.set_experiment(mlflow_experiment_name)

2023/10/05 01:22:16 INFO mlflow.tracking.fluent: Experiment with name 'rlhf_step2_reward/2023-10-05 01:22:16.267979' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/ubuntu/pykoi/example/rlhf/mlflow/mlruns/529075251471367022', creation_time=1696468936281, experiment_id='529075251471367022', last_update_time=1696468936281, lifecycle_stage='active', name='rlhf_step2_reward/2023-10-05 01:22:16.267979', tags={}>

In [7]:
# Set pykoi parameters.
reward_model_path = "elinas/llama-7b-hf-transformers-4.29"
dataset_type = "local_db"
trained_model_path = "./models/rlhf_step2_rw"

In [8]:
# Manually log pykoi parameters into mlflow. Torch level parameters are automatically logged.
mlflow.log_param("pykoi_reward_model_path", reward_model_path)
mlflow.log_param("pykoi_dataset_type", dataset_type)
mlflow.log_param("pykoi_trained_model_path", trained_model_path)

'./models/rlhf_step2_rw'

In [ ]:
# Run supervised finetuning
# Training metrics are automatically logged into mlflow.
config = RLHFConfig(reward_model_path=reward_model_path, 
                          dataset_type=dataset_type,
                          )
rlhf_step2_rft = RewardFinetuning(config)
rlhf_step2_rft.train_and_save(trained_model_path)


In [ ]:
# Save the trained reward model and input into mlflow artifacts.
mlflow.log_artifacts(trained_model_path)
mlflow.log_artifacts(input_root)

In the terminal, run
```
mlflow ui
```
and go to http://127.0.0.1:5000 in the browser to view the experiment in the UI.